In [3]:
import qiime2

import pandas as pd
from skbio import OrdinationResults
from warnings import warn
from qiime2 import Metadata

import scipy
import skbio
import qiime2
import itertools
import numpy as np 
import scipy.stats as ss
from qiime2.plugins import diversity, feature_table
from ete3 import Tree #################

##################
try:
    from q2_types.tree import NewickFormat
except ImportError:
    NewickFormat = str



In [4]:
from skbio.tree import TreeNode

from io import StringIO
from skbio import read

In [11]:
len([x for x  in t2.tips()])

7

In [12]:
tree_string = '((1:1.0,(2:1.0,3:1.0):1.0):1.0,(4:1.0,((5:1.0,6:1.0):1.0,7:1.0):1.0):1.0);'
tree_string_io = StringIO(tree_string)
t2 = read(tree_string_io, format="newick", into=TreeNode)
all_leaves = t2.tips()
numleaves = len([x for x in t2.tips()])

# Matrix that tracks all haar-like vectors
# Has n-leaves rows and n-internal nodes columns
# Each column represents an internal node
# Each row represents a leaf: 
#    0 if leaf is not a descendant from internal node
#    c_L if leaf is a left descendant from internal node
#    c_R if leaf is a right descendant from internal node
sparsehaarlike = scipy.sparse.lil_matrix((numleaves, numleaves))

# Matrix that tracks l-star values
# Has n-internal-nodes rows and n-tips columns
# Entry i, j represents weighted path distance from internal node i to tip j
# This is why it is computed iteratively for internal nodes 
#       with non-tip descendants.
lilmat = scipy.sparse.lil_matrix((numleaves, numleaves)) # 2d vector of lstar0

In [14]:
###############################################  
def get_case(node):
    """ Handle different types of children
        differently. 4 possible child layouts and 
        returns string describing the case.
        Neither => neither child tip etc.
    """

    if not node.has_children():
        return 'null'

    if node.children[0].has_children():
        if node.children[1].has_children():
            return 'neither'
        else:
            return 'right'
    else:
        if node.children[1].has_children():
            return 'left'
        else:
            return 'both'


###############################################
tip_index = {t: i for i, t in enumerate(t2.tips())}
nontip_index = {t: i for i, t in enumerate(t2.non_tips())}

fn_tip = lambda n: [tip_index[n]] if n.is_tip() else []
fn_case = lambda n: [get_case(n)] if not n.is_tip() else []

# Cache attr stores something for each child I think
t2.cache_attr(fn_tip, 'tip_names')
t2.cache_attr(fn_case, 'case')

In [15]:
###############################################
def handle_both(node, lilmat, sparsehaarlike, i):

    child = node.children
    lstar = np.zeros((numleaves, 1))
    index0 = child[0].tip_names
    index1 = child[1].tip_names

    lstar[index0] = child[0].length
    lstar[index1] = child[1].length
    lilmat[i] = np.transpose(lstar)
    haarvec = np.zeros((numleaves,1))
    haarvec[index0] = 1/np.sqrt(2)
    haarvec[index1] = -1/np.sqrt(2)
    sparsehaarlike[i] = np.transpose(haarvec)

    return lilmat, sparsehaarlike

In [42]:
def handle_left(node, lilmat, sparsehaarlike, i):

    child = node.children
    index = nontip_index[child[1]]

    lstar0 = np.zeros((numleaves, 1))
    lstar1 = np.transpose(lilmat[index].todense())


    index0 = child[0].tip_names
    index1 = child[1].tip_names

    lstar0[index0] = lstar0[index0] + len(child[0].tip_names) * child[0].length
    lstar1[index1] = lstar1[index1] + len(child[1].tip_names) * child[1].length # child[1].dist
    
    lilmat[i] = np.transpose(lstar0) + np.transpose(lstar1)
    L1 = np.count_nonzero(lstar1)
    haarvec = np.zeros((numleaves, 1))
    
    haarvec[index0] = np.sqrt(L1/(L1+1))
    haarvec[index1] = -np.sqrt(1/(L1*(L1+1)))
    sparsehaarlike[i] = np.transpose(haarvec)

    return lilmat, sparsehaarlike

In [49]:
def handle_right(node, lilmat, sparsehaarlike, i):

    child = node.children
    index = nontip_index[child[0]]

    lstar0 = np.transpose(lilmat[index].todense())
    lstar1 = np.zeros((numleaves, 1))

    index0 = child[0].tip_names
    index1 = child[1].tip_names

    lstar0[index0] = lstar0[index0] + len(child[0].tip_names) * child[0].length
    lstar1[index1] = lstar1[index1] + len(child[1].tip_names) * child[1].length
    
    lilmat[i] = np.transpose(lstar0) + np.transpose(lstar1)

    L0 = np.count_nonzero(lstar0)
    haarvec = np.zeros((numleaves, 1))
    
    haarvec[index0] = np.sqrt(1/(L0*(L0+1)))
    haarvec[index1] = -np.sqrt(L0/((L0+1)))
    sparsehaarlike[i] = np.transpose(haarvec)

    return lilmat, sparsehaarlike

In [62]:
def handle_neither(node, lilmat, sparsehaarlike, i):

    child = node.children
    index0 = nontip_index[child[0]]
    index1 = nontip_index[child[1]]

    lstar0 = np.transpose(lilmat[index0].todense())
    lstar1 = np.transpose(lilmat[index1].todense())

    index00 = child[0].tip_names
    index11 = child[1].tip_names

    print(index0)
    print(index00)
    print(child[0].tip_names)
    print(child[0].length)

    lstar0[index00] = lstar0[index00] + len(child[0].tip_names) * child[0].length
    lstar1[index11] = lstar1[index11] + len(child[1].tip_names) * child[1].length
    
    lilmat[i] = np.transpose(lstar0) + np.transpose(lstar1)

    L0 = np.count_nonzero(lstar0)
    L1 = np.count_nonzero(lstar1)

    haarvec = np.zeros((numleaves, 1))
    
    haarvec[index00] = np.sqrt(L1/(L0*(L0+L1)))
    haarvec[index11] = - np.sqrt(L0/(L1*(L0+L1)))

    sparsehaarlike[i] = np.transpose(haarvec)

    return lilmat, sparsehaarlike

In [63]:
sparsehaarlike = scipy.sparse.lil_matrix((numleaves, numleaves))
lilmat = scipy.sparse.lil_matrix((numleaves, numleaves)) # 2d vector of lstar0

i = 0
node = get_node_from_ind(t2, i)
# print(node)
# print(get_case(node))
lilmat, sparsehaarlike = handle_both(node, lilmat, 
                                             sparsehaarlike, i)
# print(lilmat.todense())
# print(sparsehaarlike.todense())

i = 1 
node = get_node_from_ind(t2, i)
# print(get_case(node))

lilmat, sparsehaarlike = handle_left(node, lilmat, sparsehaarlike, i)

# print(lilmat.todense())
# print(sparsehaarlike.todense())

i = 2
node = get_node_from_ind(t2, i)
# print(get_case(node))
lilmat, sparsehaarlike = handle_both(node, lilmat, sparsehaarlike, i)

# print(lilmat.todense())
# print(sparsehaarlike.todense())

i = 3
node = get_node_from_ind(t2, i)
# print(get_case(node))
lilmat, sparsehaarlike = handle_right(node, lilmat, sparsehaarlike, i)
# print(lilmat.todense())
# print(sparsehaarlike.todense())

i = 4
node = get_node_from_ind(t2, i)
# print(get_case(node))
lilmat, sparsehaarlike = handle_left(node, lilmat, sparsehaarlike, i)
# print(lilmat.todense())
# print(sparsehaarlike.todense())

i = 5
node = get_node_from_ind(t2, i)
print(get_case(node))
lilmat, sparsehaarlike = handle_neither(node, lilmat, sparsehaarlike, i)

print(lilmat.todense())
print(sparsehaarlike.todense())




neither
1
[0, 1, 2]
[0, 1, 2]
1.0
[[ 0.  1.  1.  0.  0.  0.  0.]
 [ 1.  3.  3.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  3.  3.  1.]
 [ 0.  0.  0.  1.  6.  6.  4.]
 [ 4.  6.  6.  5. 10. 10.  8.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
[[ 0.          0.70710678 -0.70710678  0.          0.          0.
   0.        ]
 [ 0.81649658 -0.40824829 -0.40824829  0.          0.          0.
   0.        ]
 [ 0.          0.          0.          0.          0.70710678 -0.70710678
   0.        ]
 [ 0.          0.          0.          0.          0.40824829  0.40824829
  -0.81649658]
 [ 0.          0.          0.          0.8660254  -0.28867513 -0.28867513
  -0.28867513]
 [ 0.43643578  0.43643578  0.43643578 -0.32732684 -0.32732684 -0.32732684
  -0.32732684]
 [ 0.          0.          0.          0.          0.          0.
   0.        ]]


In [23]:
child = test_node.children
index = nontip_index[child[1]] # not sure about this

lstar0 = np.zeros((numleaves, 1))
lstar1 = np.transpose(lilmat[index].todense())


index0 = child[0].tip_names
index1 = child[1].tip_names

# lstar0[index0] = child[0].length
lstar0[index0] = lstar0[index0] + len(child[0].tip_names) * child[0].length
lstar1[index1] = lstar1[index1] + len(child[1].tip_names) * child[1].length # child[1].dist

lilmat[i] = np.transpose(lstar0) + np.transpose(lstar1)
L1 = np.count_nonzero(lstar1)
haarvec = np.zeros((numleaves, 1))

haarvec[index0] = np.sqrt(L1/(L1+1))
haarvec[index1] = - np.sqrt(1/(L1*(L1+1)))
sparsehaarlike[i] = np.transpose(haarvec)

index1

[1, 2]

In [18]:

for i,n in enumerate(t2.non_tips(include_self=True)):
    print(i, n.tip_names)


####### TESTS FOR GET_CASE DONE BOOM ###############
for i,node in enumerate(t2.non_tips(include_self=True)):

    if i == 0:
        assert(get_case(node) == 'both')
    if i == 1:
        assert(get_case(node) == 'left')
        test_node = node
        # print(nontip_index[node.children[1]])
    if i == 2:
        assert(get_case(node) == 'both')
    if i == 3:
        assert(get_case(node) == 'right')
    if i == 4:
        assert(get_case(node) == 'left')
    if i == 5:
        assert(get_case(node) == 'neither')

0 [1, 2]
1 [0, 1, 2]
2 [4, 5]
3 [4, 5, 6]
4 [3, 4, 5, 6]
5 [0, 1, 2, 3, 4, 5, 6]


In [28]:
def get_node_from_ind(t2, node_ind):
    """ Helper function for testing.
        Returns i'th non-tip node. """    
    for i,n in enumerate(t2.non_tips(include_self=True)):
        if i == node_ind:
            return n

In [ ]:
test_node =

In [5]:
# # Format = 1: flexible with internal node names

# treefile = "Sparsify-Ultrametric/97_otus_unannotated.tree"
# t = Tree(treefile,format=1) 
# # This is literally the perfect example of the whole "how to bake a pie" irony
# # How do you bake a pie? First you make the crust, then filling, then bake
# # What if someone gives you the crust? You throw it away and follow the directions above.
# # There is a way to import as a NewickFormat tree, see the #### above
# t.get_closest_leaf()

In [78]:
def find_matching_index(list1, list2):

    inverse_index = { element: index for index, element in enumerate(list1) }
    print(inverse_index)
    return [(index)
        for index, element in enumerate(list2) if element in inverse_index]

In [4]:
tree_string = '((1:1.0,(2:1.0,3:1.0):1.0):1.0,(4:1.0,((5:1.0,6:1.0):1.0,7:1.0):1.0):1.0);'
t = Tree(tree_string)

# return dictionary of node instances, 
# allows quick access to node attributes without traversing the tree
node2leaves = t.get_cached_content()
numleaves = len(t) 

# Initialize row-based list of lists sparse matrix to store Haar-like vectors
# List of lists format
sparsehaarlike = scipy.sparse.lil_matrix((numleaves, numleaves))
all_leaves = t.get_leaves()
mastersplit = t.children
lilmat = scipy.sparse.lil_matrix((numleaves, numleaves))

In [ ]:
sparsehaarlike = scipy.sparse.lil_matrix((numleaves, numleaves))

In [81]:
# node2leaves is a dictionary with each node defined as all of its leaf children
# find matching index is only called with a node and all leaves

{Tree node '1' (0x7f90f6a9f6a): {Tree node '1' (0x7f90f6a9f6a)},
 Tree node '2' (0x7f90de5103d): {Tree node '2' (0x7f90de5103d)},
 Tree node '3' (0x7f90de51034): {Tree node '3' (0x7f90de51034)},
 Tree node '' (0x7f90f6a9f5e): {Tree node '2' (0x7f90de5103d),
  Tree node '3' (0x7f90de51034)},
 Tree node '' (0x7f90de481fd): {Tree node '1' (0x7f90f6a9f6a),
  Tree node '2' (0x7f90de5103d),
  Tree node '3' (0x7f90de51034)},
 Tree node '4' (0x7f90de510d0): {Tree node '4' (0x7f90de510d0)},
 Tree node '5' (0x7f90de52f04): {Tree node '5' (0x7f90de52f04)},
 Tree node '6' (0x7f90de52f4c): {Tree node '6' (0x7f90de52f4c)},
 Tree node '' (0x7f90de52feb): {Tree node '5' (0x7f90de52f04),
  Tree node '6' (0x7f90de52f4c)},
 Tree node '7' (0x7f90de52f31): {Tree node '7' (0x7f90de52f31)},
 Tree node '' (0x7f90de510ca): {Tree node '5' (0x7f90de52f04),
  Tree node '6' (0x7f90de52f4c),
  Tree node '7' (0x7f90de52f31)},
 Tree node '' (0x7f90f1bf17c): {Tree node '4' (0x7f90de510d0),
  Tree node '5' (0x7f90de52f

In [99]:
f2 = lambda n: [get_case(n)] if n.is_root() else []

t2.cache_attr(f2, 'case')

In [7]:
# USE SKBIO INSTEAD
tree_string = '((1:1.0,(2:1.0,3:1.0):1.0):1.0,(4:1.0,((5:1.0,6:1.0):1.0,7:1.0):1.0):1.0);'
tree_string_io = StringIO(tree_string)
t2 = read(tree_string_io, format="newick", into=TreeNode)

all_leaves = t2.tips()
sparsehaarlike = scipy.sparse.lil_matrix((numleaves, numleaves))
lilmat = scipy.sparse.lil_matrix((numleaves, numleaves)) # 2d vector of lstar0

# This does the same thing as find_matching_index and get_cached_content
# CORRECT
tip_index = {t: i for i, t in enumerate(t2.tips())}
nontip_index = {t: i for i, t in enumerate(t2.non_tips())}


fn_tip = lambda n: [tip_index[n]] if n.is_tip() else []
fn_case = lambda n: [get_case(n)] if not n.is_tip() else []

# Cache attr stores something for each child I think
t2.cache_attr(fn_tip, 'tip_names')
t2.cache_attr(fn_case, 'case')

traversal = t2.postorder(include_self=True)
for n in traversal:
    print(f'Node name: {n.name}, cache: {n.tip_names}')
    print(n.case)
    print('   ')

NameError: name 'numleaves' is not defined

In [84]:
for n in t2.non_tips():
    print(nontip_index[n])

0
1
2
3
4


In [ ]:
traversal = t2.non_tips(include_self=True)
for i, node in enumerate(traversal):

    case = get_case(node)

    if case == 'neither':
        print('n')
    if case == 'both':
        print('b')
    if case == 'left':
        print('l')
    if case == 'right':
        print('r')

    print(node.tip_names)
    print('  ')

    

In [6]:
def get_case(node):
    """ Handle different types of children
        differently. 4 possible child layouts and 
        returns string describing the case.
        Neither => neither child tip etc.
    """

    if not node.has_children():
        return 'null'

    if node.children[0].has_children():
        if node.children[1].has_children():
            return 'neither'
        else:
            return 'right'
    else:
        if node.children[1].has_children():
            return 'left'
        else:
            return 'both'

In [103]:
for n in t2.non_tips(include_self=True):
    print(n.tip_names, get_case(n))

    

[1, 2] both
[0, 1, 2] left
[4, 5] both
[4, 5, 6] right
[3, 4, 5, 6] left
[0, 1, 2, 3, 4, 5, 6] neither


In [ ]:
# I had an error of "TreeNode object has no pos"
# Resolution: I had added the continue break too early on in code
# ordering of nodes in post order traversal

# Error of list index out of range for lilmat
# Move i+= to the end of the code
i = 0

# Traverse non_tips in postorder
traversal = t2.non_tips(include_self=True)
for i,node in enumerate(traversal):

    pos = node.tip_names
    print(i,pos)
    
    # node.add_features(pos=pos) # store indices of leaves under each internal node
    # node.add_features(loc=i) # Add node index to node features
    
    # if node.is_tip(): continue # only iterate over internal nodes

    case = get_case(node)

    # Both children are leaves
    # if len(node2leaves[node]) == 2:

    if case == 'both_child_tip':

        def handle_both(node, lilmat, sparsehaarlike, i):

            child = node.children
            lstar = np.zeros((numleaves, 1))

            # num leaves that descend from tip is 1
            index0 = child[0].tip_names
            index1 = child[1].tip_names
            lstar[index0] = child[0].length * 1.0
            lstar[index1] = child[1].length * 1.0 
            lilmat[i] = np.transpose(lstar)

            haarvec = np.zeros((numleaves,1))
            haarvec[index0] = 1/np.sqrt(2)
            haarvec[index1] = -1/np.sqrt(2)
            sparsehaarlike[i] = np.transpose(haarvec)

            return lilmat, sparsehaarlike

        lilmat, sparsehaarlike = handle_both(node, lilmat, 
                                             sparsehaarlike, i)
        

    elif case == 'left_child_tip':
        
        def handle_left(node, lilmat, sparsehaarlike, i):

            child = node.children
            lstar0 = np.zeros((numleaves, 1))
            index0 = child[0].tip_names
            lstar0[index0] = child[0].length * 1.0

            index = nontip_index[child[1]] # not sure about this
            
            # vector representing the right child internal node
            lstar1 = np.transpose(lilmat[index].todense())

            # lstar1 is indexed by internal nodes


            index1 = child[1].tip_names
            lstar1[index1] = lstar1[index1] + len(child[1].children) * child[1].length
            lilmat[i] = np.transpose(lstar0) + np.transpose(lstar1)
            L1 = np.count_nonzero(lstar1)
            haarvec = np.zeros((numleaves, 1))
            
            haarvec[index0] = np.sqrt(L1/(L1+1))
            haarvec[index1] = -np.sqrt(1/(L1*(L1+1)))
            sparsehaarlike[i] = np.transpose(haarvec)

            return lilmat, sparsehaarlike

        lilmat, sparsehaarlike = handle_left(node, lilmat, sparsehaarlike, i)

    #     # Right child is a leaf
    #     elif len(node2leaves[child[1]]) == 1:
    #         lstar1 = np.zeros((numleaves, 1))
    #         index1 = child[1].pos
    #         lstar1[index1] = child[1].dist
    #         index = child[0].loc
    #         lstar0 = np.transpose(lilmat[index].todense())
    #         index0 = child[0].pos
    #         lstar0[index0] = lstar0[index0] + len(child[0])*child[0].dist
    #         lilmat[i] = np.transpose(lstar1) + np.transpose(lstar0)
    #         L0 = np.count_nonzero(lstar0)
            
    #         haarvec = np.zeros((numleaves,1))
    #         haarvec[index0] = np.sqrt(1/(L0*(L0+1)))
    #         haarvec[index1] = -np.sqrt(L0/((L0+1)))
    #         sparsehaarlike[i] = np.transpose(haarvec)
        
    #     # Both children are leaves?
    #     else:
    #         index0 = child[0].loc
    #         index1 = child[1].loc
            
    #         lstar0 = np.transpose(lilmat[index0].todense())
    #         lstar1 = np.transpose(lilmat[index1].todense())
            
    #         index00 = child[0].pos
    #         lstar0[index00] = lstar0[index00] + len(child[0])*child[0].dist
    #         index11 = child[1].pos
    #         lstar1[index11] = lstar1[index11]+len(child[1])*child[1].dist
    #         lilmat[i] = np.transpose(lstar0)+np.transpose(lstar1)
    #         L0 = np.count_nonzero(lstar0)
    #         L1 = np.count_nonzero(lstar1)
            
    #         haarvec = np.zeros((numleaves, 1))
    #         haarvec[index00] = np.sqrt(L1/(L0*(L0+L1)))
    #         haarvec[index11] = -np.sqrt(L0/(L1*(L0+L1)))
            
    #         sparsehaarlike[i] = np.transpose(haarvec)
    #     i+=1

In [30]:

for i,n in enumerate(t2.non_tips(include_self=True)):
    print(i, n.tip_names)

####### TESTS FOR GET_CASE DONE BOOM ###############
for i,node in enumerate(t2.non_tips(include_self=True)):

    if i == 0:
        assert(get_case(node) == 'both')
        test_node = node
    if i == 1:
        assert(get_case(node) == 'left')
        # test_node = node
        # print(nontip_index[node.children[1]])
    if i == 2:
        assert(get_case(node) == 'both')
    if i == 3:
        assert(get_case(node) == 'right')
    if i == 4:
        assert(get_case(node) == 'left')
    if i == 5:
        assert(get_case(node) == 'neither')

0 [1, 2]
1 [0, 1, 2]
2 [4, 5]
3 [4, 5, 6]
4 [3, 4, 5, 6]
5 [0, 1, 2, 3, 4, 5, 6]


In [65]:
lilmat[1].todense()

matrix([[0., 0., 0., 0., 0., 0., 0.]])

In [67]:
child = test_node.children
lstar = np.zeros((numleaves, 1))
index0 = child[0].tip_names
index1 = child[1].tip_names

print(lstar, index0, index1)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] [0] [1, 2]


In [71]:
child
index = nontip_index[child[1]]
index

KeyError: <TreeNode, name: unnamed, internal node count: 0, tips count: 2>

False

In [43]:
lstar[index0] = child[0].length
lstar[index1] = child[1].length

lilmat[0] = np.transpose(lstar)
lilmat


<7x7 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in List of Lists format>

In [57]:
def get_node_from_ind(t2, node_ind):
    """ Helper function for testing.
        Returns i'th non-tip node. """    
    for i,n in enumerate(t2.non_tips(include_self=True)):
        if i == node_ind:
            return n

sparsehaarlike = scipy.sparse.lil_matrix((numleaves, numleaves))
lilmat = scipy.sparse.lil_matrix((numleaves, numleaves))

sparsehaarlike, lilmat = handle_both(node, lilmat, sparsehaarlike, i)

haarvec = np.zeros((numleaves,1))
haarvec[index0] = 1/np.sqrt(2)
haarvec[index1] = -1/np.sqrt(2)
sparsehaarlike[0] = np.transpose(haarvec)

# shl = sparse haar like
val = 1/np.sqrt(2)
true_shl_node0 = [[0, val, -val, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0, 0, 0]]
true_shl_node0 = scipy.sparse.lil_matrix(true_shl_node0)       

# use numpy method to see if they are all close
np.allclose(true_shl_node0.A, sparsehaarlike.A)

/var/folders/pv/hg29j3j512d8wd_qw_8g3msw0000gn/T/ipykernel_3571/2898193018.py:17: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  true_shl_node0 == sparsehaarlike


True

TypeError: 'generator' object is not subscriptable

In [45]:
sparsehaarlike.data

array([list([0.7071067811865475, -0.7071067811865475]), list([]),
       list([]), list([]), list([]), list([]), list([])], dtype=object)

In [24]:
assert(get_case(node) == 'both')

In [ ]:
all_leaves = t.get_leaves()


In [ ]:
leaf_index = {l.name:i for i,l in enumerate(all_leaves)}

In [ ]:
# return dictionary of node instances, 
# allows quick access to node attributes without traversing the tree
node2leaves = t.get_cached_content()
numleaves = len(t) 

# Initialize row-based list of lists sparse matrix to store Haar-like vectors
# List of lists format
sparsehaarlike = scipy.sparse.lil_matrix((numleaves, numleaves))
all_leaves = t.get_leaves()
mastersplit = t.children
lilmat = scipy.sparse.lil_matrix((numleaves, numleaves))

In [ ]:
node2leaves

In [ ]:
def is_leaf_fn(node):
    print(node, node.is_leaf())
    return node.is_leaf()


traversal = t.traverse("postorder", is_leaf_fn)
for node in traversal:
    print('xxxxxxxx')

In [ ]:
traversal = t.traverse("postorder")
for node in traversal:

    pos = find_matching_index(node, all_leaves)
    print(pos)
    print('   ')

In [ ]:
for i, node in enumerate(node):
    print(i, node)

In [ ]:
# I had an error of "TreeNode object has no pos"
# Resolution: I had added the continue break too early on in code
# ordering of nodes in post order traversal

# Error of list index out of range for lilmat
# Move i+= to the end of the code
i = 0
traversal = t.traverse("postorder")
for node in traversal:

    print(i,node)
    pos = find_matching_index(node, all_leaves)
    
    node.add_features(pos=pos) # store indices of leaves under each internal node
    node.add_features(loc=i) # Add node index to node features
    
    if node.is_leaf(): continue # only iterate over internal nodes

    # Both children are leaves
    if len(node2leaves[node]) == 2:
        child = node.children
        lstar = np.zeros((numleaves, 1))
        index0 = child[0].pos
        index1 = child[1].pos
        lstar[index0] = child[0].dist
        lstar[index1] = child[1].dist
        lilmat[i] = np.transpose(lstar)
        haarvec = np.zeros((numleaves,1))
        haarvec[index0] = 1/np.sqrt(2)
        haarvec[index1] = -1/np.sqrt(2)
        sparsehaarlike[i] = np.transpose(haarvec)

    else:
        
        child = node.children

        # Left child is a leaf
        if len(node2leaves[child[0]]) == 1:
            lstar0 = np.zeros((numleaves, 1))
            index0 = child[0].pos
            lstar0[index0] = child[0].dist
            index = child[1].loc
            lstar1 = np.transpose(lilmat[index].todense())
            index1 = child[1].pos                
            lstar1[index1] = lstar1[index1] + len(child[1])*child[1].dist
            lilmat[i] = np.transpose(lstar0) + np.transpose(lstar1)
            L1 = np.count_nonzero(lstar1)
            haarvec = np.zeros((numleaves, 1))
            
            haarvec[index0] = np.sqrt(L1/(L1+1))
            haarvec[index1] = -np.sqrt(1/(L1*(L1+1)))
            sparsehaarlike[i] = np.transpose(haarvec)

        # Right child is a leaf
        elif len(node2leaves[child[1]]) == 1:
            lstar1 = np.zeros((numleaves, 1))
            index1 = child[1].pos
            lstar1[index1] = child[1].dist
            index = child[0].loc
            lstar0 = np.transpose(lilmat[index].todense())
            index0 = child[0].pos
            lstar0[index0] = lstar0[index0] + len(child[0])*child[0].dist
            lilmat[i] = np.transpose(lstar1) + np.transpose(lstar0)
            L0 = np.count_nonzero(lstar0)
            
            haarvec = np.zeros((numleaves,1))
            haarvec[index0] = np.sqrt(1/(L0*(L0+1)))
            haarvec[index1] = -np.sqrt(L0/((L0+1)))
            sparsehaarlike[i] = np.transpose(haarvec)
        
        # Both children are leaves?
        else:
            index0 = child[0].loc
            index1 = child[1].loc
            
            lstar0 = np.transpose(lilmat[index0].todense())
            lstar1 = np.transpose(lilmat[index1].todense())
            
            index00 = child[0].pos
            lstar0[index00] = lstar0[index00] + len(child[0])*child[0].dist
            index11 = child[1].pos
            lstar1[index11] = lstar1[index11]+len(child[1])*child[1].dist
            lilmat[i] = np.transpose(lstar0)+np.transpose(lstar1)
            L0 = np.count_nonzero(lstar0)
            L1 = np.count_nonzero(lstar1)
            
            haarvec = np.zeros((numleaves, 1))
            haarvec[index00] = np.sqrt(L1/(L0*(L0+L1)))
            haarvec[index11] = -np.sqrt(L0/(L1*(L0+L1)))
            
            sparsehaarlike[i] = np.transpose(haarvec)
        i+=1

In [ ]:
lilmat.data

In [ ]:
i=0 #ordering of nodes in post order traversal
for node in t.traverse("postorder", is_leaf_fn):
    
    node.add_features(pos=find_matching_index(node,all_leaves)) # store indices of leaves under each internal node
    veclen=len(node2leaves[node])
    print(i)
    
    if not node.is_leaf():
        node.add_features(loc=i) #add node index to node features
        if veclen==2:
            child=node.children
            lstar=np.zeros((numleaves,1))
            index0=child[0].pos
            index1=child[1].pos
            lstar[index0]=child[0].dist
            lstar[index1]=child[1].dist
            lilmat[i]=np.transpose(lstar)
            haarvec=np.zeros((numleaves,1))
            haarvec[index0]=1/np.sqrt(2)
            haarvec[index1]=-1/np.sqrt(2)
            sparsehaarlike[i]=np.transpose(haarvec)
            i=i+1
        else:
            child=node.children
            if len(node2leaves[child[0]])==1:
                lstar0=np.zeros((numleaves,1))
                index0=child[0].pos
                lstar0[index0]=child[0].dist
                index=child[1].loc
                lstar1=np.transpose(lilmat[index].todense())
                index1=child[1].pos                
                lstar1[index1]=lstar1[index1]+len(child[1])*child[1].dist
                lilmat[i]=np.transpose(lstar0)+np.transpose(lstar1)
                L1=np.count_nonzero(lstar1)
                haarvec=np.zeros((numleaves,1))
                haarvec[index0]=np.sqrt(L1/(L1+1))
                haarvec[index1]=-np.sqrt(1/(L1*(L1+1)))
                sparsehaarlike[i]=np.transpose(haarvec)
                i=i+1
            elif len(node2leaves[child[1]])==1:
                lstar1=np.zeros((numleaves,1))
                index1=child[1].pos
                lstar1[index1]=child[1].dist
                index=child[0].loc
                lstar0=np.transpose(lilmat[index].todense())
                index0=child[0].pos
                lstar0[index0]=lstar0[index0]+len(child[0])*child[0].dist
                lilmat[i]=np.transpose(lstar1)+np.transpose(lstar0)
                L0=np.count_nonzero(lstar0)
                haarvec=np.zeros((numleaves,1))
                haarvec[index0]=np.sqrt(1/(L0*(L0+1)))
                haarvec[index1]=-np.sqrt(L0/((L0+1)))
                sparsehaarlike[i]=np.transpose(haarvec)
                i=i+1
            else:
                index0=child[0].loc
                index1=child[1].loc
                lstar0=np.transpose(lilmat[index0].todense())
                lstar1=np.transpose(lilmat[index1].todense())
                index00=child[0].pos
                lstar0[index00]=lstar0[index00]+len(child[0])*child[0].dist
                index11=child[1].pos
                lstar1[index11]=lstar1[index11]+len(child[1])*child[1].dist
                lilmat[i]=np.transpose(lstar0)+np.transpose(lstar1)
                L0=np.count_nonzero(lstar0)
                L1=np.count_nonzero(lstar1)
                haarvec=np.zeros((numleaves,1))
                haarvec[index00]=np.sqrt(L1/(L0*(L0+L1)))
                haarvec[index11]=-np.sqrt(L0/(L1*(L0+L1)))
                sparsehaarlike[i]=np.transpose(haarvec)
                i=i+1

In [ ]:
# CODE GRAVEYARD



t2 = read(StringIO(tree_string), 
          format="newick", into=TreeNode)

node_index = {l: i for i, l in enumerate(t2.tips())}
f = lambda n: [node_index[n]] if n.is_tip() else []
f2 = lambda n: [n] 


t2.cache_attr(f, 'tip_names')

traversal = t2.postorder(include_self=True)
for n in traversal:
    print("Node name: %s, cache: %r" % (n.name, n.tip_names))
    print('   ')




treefile = "Sparsify-Ultrametric/tree.qza"

tree = qiime2.Artifact.load(treefile)

try:
    from q2_types.tree import NewickFormat
except ImportError:
    NewickFormat = str



